# Factorisation de matrice
## Première approche "naïve"

Nous implémentons un modèle simple de filtrage collaboratif, basé sur la factorisation de matrice. 

### Installation et import des bibliothèques

In [2]:
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install torch torchvision torchaudio
!{sys.executable} -m pip install pytorch-lightning


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
import numpy as np
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.nn.functional as F
from pytorch_lightning.loggers import TensorBoardLogger

### Récupération des csv, et description des `DataFrames`

In [2]:
genome_scores = pd.read_csv("../data/raw/genome-scores.csv")
genome_tags = pd.read_csv("../data/raw/genome-tags.csv")
links = pd.read_csv("../data/raw/links.csv")
movies = pd.read_csv("../data/raw/movies.csv")
ratings = pd.read_csv("../data/raw/ratings.csv")
tags = pd.read_csv("../data/raw/tags.csv")

In [5]:
l = [genome_scores, genome_tags, links, movies, ratings, tags]

for elt in l: 
    display(elt.head())
    display(elt.info())


,movieId,tagId,relevance
0,1,1,0.02500
1,1,2,0.02500
2,1,3,0.05775
3,1,4,0.09675
4,1,5,0.14675


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11709768 entries, 0 to 11709767
Data columns (total 3 columns):
 #   Column     Dtype  
---  ------     -----  
 0   movieId    int64  
 1   tagId      int64  
 2   relevance  float64
dtypes: float64(1), int64(2)
memory usage: 268.0 MB


None

,tagId,tag
0,1,007
1,2,007 (series)
2,3,18th century
3,4,1920s
4,5,1930s


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1128 entries, 0 to 1127
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tagId   1128 non-null   int64 
 1   tag     1128 non-null   object
dtypes: int64(1), object(1)
memory usage: 17.8+ KB


...

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000263 entries, 0 to 20000262
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 610.4 MB


None

,userId,movieId,tag,timestamp
0,18,4141,Mark Waters,1240597180
1,65,208,dark hero,1368150078
2,65,353,dark hero,1368150079
3,65,521,noir thriller,1368149983
4,65,592,dark hero,1368150078


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 465564 entries, 0 to 465563
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   userId     465564 non-null  int64 
 1   movieId    465564 non-null  int64 
 2   tag        465548 non-null  object
 3   timestamp  465564 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 14.2+ MB


None

In [6]:
print(f"Il y a {ratings.movieId.nunique()} films dans notre jeu de données.")
print(f"Il y a {ratings.userId.nunique()} utilisateurs dans notre jeu de données.")

Il y a 26744 films dans notre jeu de données.
Il y a 138493 utilisateurs dans notre jeu de données.


### Ajout de la date sur les `DataFrames` **`ratings`** et **`tags`**

In [6]:
ratings2 = ratings.copy()
tags2 = tags.copy()

ratings2["date"] = pd.to_datetime(ratings2["timestamp"], unit = "s")
tags2["date"] = pd.to_datetime(tags2["timestamp"], unit = "s")

display(ratings2.head())
display(tags2.head())

,userId,movieId,rating,timestamp,date
0,1,2,3.5,1112486027,2005-04-02 23:53:47
1,1,29,3.5,1112484676,2005-04-02 23:31:16
2,1,32,3.5,1112484819,2005-04-02 23:33:39
3,1,47,3.5,1112484727,2005-04-02 23:32:07
4,1,50,3.5,1112484580,2005-04-02 23:29:40


,userId,movieId,tag,timestamp,date
0,18,4141,Mark Waters,1240597180,2009-04-24 18:19:40
1,65,208,dark hero,1368150078,2013-05-10 01:41:18
2,65,353,dark hero,1368150079,2013-05-10 01:41:19
3,65,521,noir thriller,1368149983,2013-05-10 01:39:43
4,65,592,dark hero,1368150078,2013-05-10 01:41:18


In [7]:
print("Étendue des ratings : de ", ratings2.date.min(), " à ", ratings2.date.max())
print("Étendue des tags : de", tags2.date.min(), " à ", tags2.date.max())

Étendue des ratings : de  1995-01-09 11:46:44  à  2015-03-31 06:40:02
Étendue des tags : de 2005-12-24 13:00:10  à  2015-03-31 03:09:12


### Construction de la matrice d'interaction
Observons le nombre de modalités dans les variables userId et movieId.

In [8]:
ratings.nunique()

userId         138493
movieId         26744
rating             10
timestamp    15351121
dtype: int64

L'objectif est de construire une matrice $A\in \mathbb{R}^{m*n}$ dont les éléments représenteraient les notes explicites, et où : 
* $m$ est le nombre total d'utilisateurs (égal à **`ratings.userId.nunique()`**)
* $n$ est le nombre total de films (égal à **`ratings.movieId.nunique()`**)


Avec les nombres bruts obtenus, on aurait une matrice de taille $138~493 * 26~744$, ce qui correspond à $3.7$ milliards d'éléments. Dû à des limitations techniques, nous sommes contraints de réduire notre jeu de données de manière significative.

In [9]:
try : 
    ratings.copy().pivot(index = "userId", columns = "movieId", values = "rating")
except ValueError: # un overflow survient, on obtient une dimension négative.
    print("La transformation a échoué, les dimensions d'entrée sont trop larges")

c:\Users\pc-04\Documents\Template_MLOps_movie_recommandation\.venv\lib\site-packages\pandas\core\reshape\reshape.py:125: RuntimeWarning: overflow encountered in scalar multiply
  num_cells = num_rows * num_columns


La transformation a échoué, les dimensions d'entrée sont trop larges


In [10]:
# On réduit le jeu de données via sample (on ne garde que 0.1% du jeu de données), puis on utilise pivot() pour générer la matrice d'interactions
interactions = ratings.sample(frac = 0.001).reset_index(drop = True).pivot(index = "userId", columns = "movieId", values = "rating")

display(interactions)

# Observation de la disparité des données
print(f"Il y a {interactions.isna().sum().sum()} valeurs manquantes.\n\
Au total, il y a {interactions.size} éléments dans la DataFrame interactions\n\
Il y a donc {(interactions.isna().sum().sum()/interactions.size)*100:.3f}% de valeurs manquantes\n")
interactions.loc[:, 0:20].info()

movieId,1,2,3,4,5,6,7,8,9,10,...,115617,115831,116977,117176,117590,118700,118997,126283,129415,130382
userId,,,,,,,,,,,,,,,,,,,,,
11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138437,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138457,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138470,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Il y a 75529066 valeurs manquantes.
Au total, il y a 75549066 éléments dans la DataFrame interactions
Il y a donc 99.974% de valeurs manquantes

<class 'pandas.core.frame.DataFrame'>
Index: 16342 entries, 11 to 138486
Data columns (total 20 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   1       45 non-null     float64
 1   2       25 non-null     float64
 2   3       8 non-null      float64
 3   4       4 non-null      float64
 4   5       11 non-null     float64
 5   6       30 non-null     float64
 6   7       14 non-null     float64
 7   8       3 non-null      float64
 8   9       2 non-null      float64
 9   10      20 non-null     float64
 10  11      19 non-null     float64
 11  12      4 non-null      float64
 12  13      1 non-null      float64
 13  14      8 non-null      float64
 14  15      2 non-null      float64
 15  16      18 non-null     float64
 16  17      21 non-null     float64
 17  18      3 non-null      float64
 18  19  

La factorisation de matrice (_Matrix Factorization_ en anglais) est une classe de modèles de filtrage collaboratif. **L'idée essentielle est de considérer la matrice d'interactions comme étant le produit de deux matrices de dimensions inférieures à la matrice d'origine.** Cette décomposition permet de mettre en lumière les caractéristiques implicites du système.

Ainsi, on définit deux nouvelles matrices :
* la **matrice latente des utilisateurs** $P \in \mathbb{R}^{m*k}$
* la **matrice latente des films** $Q \in \mathbb{R}^{n*k}$

Que vaut le paramètre $k$ ? Appelé **taille du facteur latent**, ce dernier est strictement inférieur aux dimensions d'origine $m$ et $n$.

Le produit des matrices latentes donne une matrice $\hat{A}$ de même taille que $A$. Cette relation est donnée via l'équation suivante : 

$ \hat{A} = P*Q^{T}$, avec $\hat{A} \approx A$

Cette approximation ne fait sens que lorsque l'on compare les éléments de $\hat{A}$ avec ceux non manquants dans $A$ ; on définit ainsi l'ensemble des observations pertinentes $\kappa$ :

$\kappa = \{(u, i)~|~A_{ui}~est~connu \}$

Lors de l'entraînement du modèle, nous allons chercher à minimiser l'erreur moyenne quadratique (MSE) entre les évaluations prédites et réelles, tout en prenant en considération les biais éventuels touchant les utilisateurs comme les films. La fonction de perte à minimiser devient alors la suivante :  

$ \underset{P, Q, b}{argmin}\underset{i,j\in{\kappa}}\sum\|A_{ij} - \hat{A{ij}}\|^{2} + \lambda (\|P\|^2_F + \|Q\|_F^2 + b_u^2 + b_i^2)$

Où $\lambda$ dénote le facteur de régularisation.

Les paramètres du modèle peuvent être appris avec un algorithme usuel d'optimisation tel que la descente de gradient stochastique (SGD) ou Adam.

In [11]:
interactions = np.nan_to_num(interactions)


interactions

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

### Construction sur PyTorch

In [12]:
class MatrixFactorization(nn.Module):
    def __init__(self, embedding_dims, num_users, num_items, sparse = False, **kwargs):
        super().__init__()
        self.sparse = sparse

        self.user_embedding = nn.Embedding(num_users, embedding_dims, sparse = sparse)
        self.user_bias = nn.Embedding(num_users, 1, sparse = sparse)

        self.item_embedding = nn.Embedding(num_items, embedding_dims, sparse = sparse)
        self.item_bias = nn.Embedding(num_items, 1, sparse = sparse)

        for param in self.parameters():
            nn.init.normal_(param, std=0.01)
    
    def forward(self, user_id, item_id):
        P_u = self.user_embedding(user_id)
        b_u = self.user_bias(user_id).flatten()

        Q_i = self.item_embedding(item_id)
        b_i = self.item_bias(item_id).flatten()

        return (P_u*Q_i).sum(-1) + b_u + b_i

In [14]:
model = MatrixFactorization(5, 5, 5)

In [15]:
model.state_dict()

OrderedDict([('user_embedding.weight',
              tensor([[-1.2568e-02, -4.6730e-03, -3.3334e-03,  1.2265e-03, -1.2654e-02],
                      [ 3.1120e-03, -3.6110e-03,  1.5917e-02,  2.5636e-02, -1.6199e-03],
                      [-1.2424e-02,  2.3483e-02,  3.1398e-03,  6.0457e-03,  2.0491e-03],
                      [ 2.3703e-02, -1.1297e-03,  1.5363e-02, -4.1876e-07, -8.4021e-03],
                      [-7.7370e-03, -1.0735e-02,  1.1534e-02, -1.4830e-02,  1.7291e-03]])),
             ('user_bias.weight',
              tensor([[-0.0015],
                      [-0.0062],
                      [-0.0020],
                      [-0.0199],
                      [-0.0085]])),
             ('item_embedding.weight',
              tensor([[ 0.0071,  0.0014, -0.0112, -0.0096,  0.0036],
                      [ 0.0082, -0.0110, -0.0076, -0.0130, -0.0089],
                      [ 0.0035,  0.0033,  0.0100, -0.0104,  0.0066],
                      [-0.0011,  0.0094, -0.0031,  0.0111,  0.008

In [27]:
ratings.loc[(ratings.userId == 1), ["userId"]]

,userId
0,1
1,1
2,1
3,1
4,1
...,...
170,1
171,1
172,1
173,1
